# Flux of Mn across boundaries

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import netCDF4 as nc
from itertools import compress
import datetime as dt
import seaborn as sns; sns.set()
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799
isize = imax-imin
jsize = jmax-jmin

# Mn model result folders:
folder_ref      = '/data/brogalla/run_storage/Mn-reference-202110/'
folder_cleanice = '/data/brogalla/run_storage/Mn-clean-ice-202110/'
folder_spm      = '/data/brogalla/run_storage/Mn-spm-202110/'

# Colors:
dirty  = '#90a1b1'
clean  = '#cee7fd'
rivers = '#519e98'
colors = ['#ccb598', '#448d90', '#739f78', '#CC8741', '#cee7fd', '#b9c1c7']

In [ ]:
years = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, \
         2016, 2017, 2018, 2019]

#### Load files:

In [ ]:
# ANHA12 grid:
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask      = np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
land_mask  = np.ma.masked_where((tmask[:,:,:] > 0.1), tmask[:,:,:]) 
lons       = np.array(mesh.variables['nav_lon'])
lats       = np.array(mesh.variables['nav_lat'])
depths     = np.array(mesh.variables['gdept_1d'])[0,:]
e1t_base   = np.array(mesh.variables['e1t'])[0,imin:imax,jmin:jmax]
e2t_base   = np.array(mesh.variables['e2t'])[0,imin:imax,jmin:jmax]
e3t        = np.array(mesh.variables['e3t_0'])[0,:,imin:imax,jmin:jmax]
e3t_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), e3t)

e1t        = np.tile(e1t_base, (50,1,1))
e2t        = np.tile(e2t_base, (50,1,1))
volume     = e1t*e2t*e3t
area_base  = e1t_base*e2t_base
volume_masked = np.ma.masked_where((tmask[:,:,:] < 0.1), volume)
area_masked   = np.ma.masked_where((tmask[0,:,:] < 0.1), area_base)

#### Functions:

In [ ]:
def time_series_pickle(year, nosed=False, rivers=False):
    folder = '/ocean/brogalla/GEOTRACES/time-series/'
    
    # Load calculated time series of fluxes:
    # Time series calculated using function: /calculations/time_series-calc.py
    if nosed:
        time_series_V1, time_series_V2, time_series_V3, time_series_V4, time_series_V5, time_series_V6,time_series_V7,\
        time_series_V8, time_series_V9, time_series_V10, time_series_V11, time_series_mn1, time_series_mn2, \
        time_series_mn3, time_series_mn4, time_series_mn5, time_series_mn6, time_series_mn7, time_series_mn8, \
        time_series_mn9, time_series_mn10, time_series_mn11 = \
            pickle.load(open(f'{folder}Mn-clean-ice-202110/time-series-{year}.pickle','rb'))  
    elif rivers:
        time_series_V1, time_series_V2, time_series_V3, time_series_V4, time_series_V5, time_series_V6,time_series_V7,\
        time_series_V8, time_series_V9, time_series_V10, time_series_V11, time_series_mn1, time_series_mn2, \
        time_series_mn3, time_series_mn4, time_series_mn5, time_series_mn6, time_series_mn7, time_series_mn8, \
        time_series_mn9, time_series_mn10, time_series_mn11 = \
            pickle.load(open(f'{folder}Mn-spm-202110/time-series-{year}.pickle','rb'))  
    else:
        time_series_V1, time_series_V2, time_series_V3, time_series_V4, time_series_V5, time_series_V6,time_series_V7,\
        time_series_V8, time_series_V9, time_series_V10, time_series_V11, time_series_mn1, time_series_mn2, \
        time_series_mn3, time_series_mn4, time_series_mn5, time_series_mn6, time_series_mn7, time_series_mn8, \
        time_series_mn9, time_series_mn10, time_series_mn11 = \
            pickle.load(open(f'{folder}Mn-reference-202110/time-series-{year}.pickle','rb'))
    
    return time_series_mn2, time_series_mn11

In [ ]:
def file_dates(year):
    #start_date and end_date are datetime objects
    start_date = dt.datetime(year,1,1)
    end_date   = dt.datetime(year,12,31)
    
    file_list1 = np.sort(os.listdir(f'{folder_ref}ANHA12_ref-{year}_20211012/'))
    file_list2 = np.sort(os.listdir('/data/brogalla/ANHA12/'))
    
    Vlist = [i[26:31]=='gridV' for i in file_list2]
    gridV_list = list(compress(file_list2, Vlist))
    dateV_list = [dt.datetime.strptime(i[14:25], "y%Ym%md%d") for i in gridV_list]
    gridV_file_list = list(compress(gridV_list, [V > start_date and V < end_date for V in dateV_list]))
    
    dates = [dt.datetime.strptime(i[14:25], "y%Ym%md%d") for i in gridV_file_list]
    
    return dates

#### Calculations:

In [ ]:
ref_in    = np.empty((len(years),72,50,79)); ref_out   = np.empty((len(years),72,50,52)); # ref
nosed_in  = np.empty((len(years),72,50,79)); nosed_out = np.empty((len(years),72,50,52)); # nosed
spm_in    = np.empty((len(years),72,50,79)); spm_out   = np.empty((len(years),72,50,52)); # rivers

# Load time series of fluxes
for i, year in enumerate(years):
    ref_in1, ref_out1     = time_series_pickle(year)
    nosed_in2, nosed_out2 = time_series_pickle(year, nosed=True)
    spm_in3, spm_out3     = time_series_pickle(year, rivers=True)
    
    ref_in[i,:,:,:]    = ref_in1*1e3 # convert from mol m3/ L s --> mol / s
    nosed_in[i,:,:,:]  = nosed_in2*1e3
    spm_in[i,:,:,:]    = spm_in3*1e3
    ref_out[i,:,:,:]   = ref_out1*1e3
    nosed_out[i,:,:,:] = nosed_out2*1e3
    spm_out[i,:,:,:]   = spm_out3*1e3

In [ ]:
dates = np.array([])
for year in range(2002, 2020):
    dates = np.append(dates, file_dates(year))

In [ ]:
# Parry channel reference run fluxes:
Parry_ref_cumsumin  = np.cumsum(np.sum(-1*ref_in, axis=(2,3)).flatten())
Parry_ref_cumsumout = np.cumsum(np.sum(-1*ref_out, axis=(2,3)).flatten())

Parry_ref_in  = np.sum(-1*ref_in, axis=(2,3)).flatten()
Parry_ref_out = np.sum(-1*ref_out, axis=(2,3)).flatten()

Parry_ref_diff = Parry_ref_out - Parry_ref_in

In [ ]:
# Parry channel clean sea ice run fluxes:
Parry_nosed_cumsumin  = np.cumsum(np.sum(-1*nosed_in, axis=(2,3)).flatten())
Parry_nosed_cumsumout = np.cumsum(np.sum(-1*nosed_out, axis=(2,3)).flatten())

Parry_nosed_in  = np.sum(-1*nosed_in, axis=(2,3)).flatten()
Parry_nosed_out = np.sum(-1*nosed_out, axis=(2,3)).flatten()

Parry_nosed_diff = Parry_nosed_out - Parry_nosed_in

In [ ]:
# Parry channel particulate riverine run fluxes:
Parry_rivers_cumsumin  = np.cumsum(np.sum(-1*(spm_in), axis=(2,3)).flatten())
Parry_rivers_cumsumout = np.cumsum(np.sum(-1*(spm_out), axis=(2,3)).flatten())

Parry_rivers_in  = np.sum(-1*(spm_in), axis=(2,3)).flatten()
Parry_rivers_out = np.sum(-1*(spm_out), axis=(2,3)).flatten()

Parry_rivers_diff = Parry_rivers_out - Parry_rivers_in

### Figures

In [ ]:
fig = plt.figure(figsize=(4.5, 3.2), dpi=300)
st  = sns.axes_style("whitegrid")

with st:
    ax = sns.lineplot(dates, Parry_rivers_in,    linewidth=0.6, color=rivers, markersize=7)
    sns.lineplot(dates,      Parry_ref_in,       linewidth=0.6, color=dirty)
    sns.lineplot(dates,      Parry_nosed_in,     linewidth=0.6, color=clean, markersize=7)
    sns.lineplot(dates,      -1*Parry_rivers_out,linewidth=0.6, color=rivers)
    sns.lineplot(dates,      -1*Parry_ref_out,   linewidth=0.6, color=dirty)
    sns.lineplot(dates,      -1*Parry_nosed_out, linewidth=0.6, color=clean)
    
    sns.lineplot(dates[0],[0], linewidth=3, color=dirty, label=r'Sediment in sea ice')
    sns.lineplot(dates[0],[0], linewidth=3, color=clean, label=r'Clean sea ice')
    sns.lineplot(dates[0],[0], linewidth=3, color=rivers, label=r'Upper bound river')

    sns.lineplot([dt.datetime(2002,1,1), dt.datetime(2020,1,1)], [0,0], c='k')
    
    ax.set_xlabel('Year', fontsize=6)
    ax.set_ylabel('Flux of Mn in/out Parry Channel [mol/s]',fontsize=6)
    ax.set(xlim=(dt.datetime(2002,1,1), dt.datetime(2020,1,1)), ylim=(-8,6))
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.spines['left'].set_linewidth(1.0);   ax.spines['left'].set_color('black');
    ax.spines['bottom'].set_linewidth(1.0); ax.spines['bottom'].set_color('black');
    ax.spines['right'].set_linewidth(1.0);  ax.spines['right'].set_color('black');
    ax.spines['top'].set_linewidth(1.0);    ax.spines['top'].set_color('black');    
    
    leg = ax.legend(loc=(0.01, 0.78), frameon=True, framealpha=0.7, fontsize=6)
    leg.get_frame().set_linewidth(0.0)
    leg.set_title('Model experiment: ',prop={'size':6})
    
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S13-flux-timeseries.png', bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S13-flux-timeseries.svg', bbox_inches='tight', dpi=300, \
            format='svg')

In [ ]:
st  = sns.axes_style("whitegrid")

with st:
    fig = plt.figure(figsize=(4.5, 3.2), dpi=300)

    ax = sns.lineplot(dates, (1-np.divide(Parry_nosed_cumsumin, Parry_ref_cumsumin))*100, \
                      linewidth=1.5, color=dirty, label=r'From Canada Basin into Parry Channel')
    sns.lineplot(dates, (1-np.divide(Parry_nosed_cumsumout, Parry_ref_cumsumout))*100, \
                 linewidth=1.5, color=dirty, label=r'Out of Parry Channel into Baffin Bay')
    ax.lines[1].set_linestyle('--')
    
    ax.set_xlabel('Year', fontsize=6)
    ax.set_ylabel('Sea ice contribution to cumulative Mn flux [%]',fontsize=6)
    ax.set(xlim=(dt.datetime(2002,1,1), dt.datetime(2020,1,1)), ylim=(0, 100))
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.spines['left'].set_linewidth(1.5);   ax.spines['left'].set_color('black');
    ax.spines['bottom'].set_linewidth(1.5); ax.spines['bottom'].set_color('black');
    ax.spines['right'].set_linewidth(1.5);  ax.spines['right'].set_color('black');
    ax.spines['top'].set_linewidth(1.5);    ax.spines['top'].set_color('black');    
    
    leg = ax.legend(loc=(0.03, 0.01), frameon=True, framealpha=0.7, fontsize=6)
    leg.get_frame().set_linewidth(0.0)
    
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S14-flux-percentage.png', bbox_inches='tight', dpi=300)
fig.savefig('/ocean/brogalla/GEOTRACES/figures/paper1-202110/S14-flux-percentage.svg', format='svg', \
                            bbox_inches='tight', dpi=300)